# 📚 SSG Epub Admin Tool - Google Colab

Công cụ quản lý sách điện tử cho SSG Epub Library

## 🚀 Hướng dẫn sử dụng:
1. Chạy từng cell theo thứ tự
2. Nhập thông tin khi được yêu cầu
3. Sử dụng URL public để truy cập admin tool

---

## 📦 Bước 1: Cài đặt packages

In [ ]:
# Cài đặt các package cần thiết
!pip install streamlit requests PyYAML pyngrok

print("✅ Đã cài đặt xong các packages!")

## 📂 Bước 2: Clone repository

In [ ]:
import os
from google.colab import userdata

# Lấy URL repository từ Colab Secrets hoặc nhập thủ công
try:
    REPO_URL = userdata.get('GITHUB_REPO_URL')
    print(f"✅ Lấy được repository URL từ secrets: {REPO_URL}")
except:
    REPO_URL = input("Nhập URL repository (ví dụ: https://github.com/username/SSGEpub.git): ")

if REPO_URL:
    # Xóa thư mục cũ nếu có
    !rm -rf /content/SSGEpub
    
    # Clone repository
    !git clone {REPO_URL} /content/SSGEpub
    
    # Chuyển đến thư mục repository
    %cd /content/SSGEpub
    
    print("✅ Đã clone repository thành công!")
    print(f"📂 Repository: {REPO_URL}")
    
    # Kiểm tra admin_tool có tồn tại không
    if os.path.exists('/content/SSGEpub/admin_tool/main.py'):
        print("✅ Tìm thấy admin_tool")
    else:
        print("⚠️ Không tìm thấy admin_tool, sẽ tạo mới")
        # Tạo admin_tool nếu chưa có
        !mkdir -p admin_tool/data
        print("📁 Đã tạo thư mục admin_tool")
else:
    print("❌ Vui lòng nhập URL repository!")
    print("💡 Tip: Thêm GITHUB_REPO_URL vào Colab Secrets để không phải nhập lại")

## 🔑 Bước 3: Cấu hình authentication

In [ ]:
from google.colab import userdata
import os

# Lấy thông tin từ Colab Secrets hoặc nhập thủ công
try:
    git_name = userdata.get('GITHUB_USERNAME')
    print(f"✅ Lấy được GitHub username từ secrets: {git_name}")
except:
    git_name = input("Nhập Git user name (mặc định: Admin Bot): ") or "Admin Bot"

try:
    git_email = userdata.get('GITHUB_EMAIL')
    print(f"✅ Lấy được GitHub email từ secrets: {git_email}")
except:
    git_email = input("Nhập Git email (mặc định: admin@ssgepub.com): ") or "admin@ssgepub.com"

# Cấu hình Git
!git config --global user.name "{git_name}"
!git config --global user.email "{git_email}"
!git config --global --add safe.directory /content/SSGEpub

print(f"✅ Đã cấu hình Git: {git_name} <{git_email}>")

# GitHub Token
try:
    github_token = userdata.get('GITHUB_TOKEN')
    os.environ['GITHUB_TOKEN'] = github_token
    print("✅ Lấy được GitHub token từ secrets!")
except:
    github_token = input("Nhập GitHub Personal Access Token (hoặc thêm vào Colab Secrets với key 'GITHUB_TOKEN'): ")
    if github_token:
        os.environ['GITHUB_TOKEN'] = github_token
        print("✅ Đã lưu GitHub token!")
    else:
        print("⚠️ Không có GitHub token. Bạn có thể cấu hình sau trong admin tool.")

print("\n💡 Tip: Để không phải nhập lại, hãy thêm vào Colab Secrets:")
print("   - GITHUB_TOKEN: Personal Access Token")
print("   - GITHUB_USERNAME: Tên GitHub của bạn")
print("   - GITHUB_EMAIL: Email GitHub của bạn")
print("   - NGROK_TOKEN: Auth token từ ngrok.com (tùy chọn)")

## 🌐 Bước 4: Setup ngrok

In [ ]:
from pyngrok import ngrok
from google.colab import userdata

# Lấy ngrok token từ secrets hoặc nhập thủ công
try:
    ngrok_token = userdata.get('NGROK_TOKEN')
    ngrok.set_auth_token(ngrok_token)
    print("✅ Lấy được ngrok token từ secrets!")
except:
    ngrok_token = input("Nhập ngrok auth token (lấy từ https://dashboard.ngrok.com/get-started/your-authtoken): ")
    if ngrok_token:
        ngrok.set_auth_token(ngrok_token)
        print("✅ Đã cấu hình ngrok!")
    else:
        print("⚠️ Không có ngrok token. Admin tool sẽ chỉ chạy local.")

print("\n📝 Lưu ý về ngrok:")
print("🎯 Ngrok tạo URL public để truy cập Streamlit app từ browser")
print("🔗 Ví dụ: https://abc123.ngrok.io -> localhost:8501 trên Colab")
print("✨ Nếu chạy local trên máy bạn thì KHÔNG CẦN ngrok!")

## 🎨 Bước 5: Cấu hình Streamlit

In [ ]:
# Tạo cấu hình Streamlit
import os
from pathlib import Path

# Tạo thư mục .streamlit
streamlit_dir = Path.home() / ".streamlit"
streamlit_dir.mkdir(exist_ok=True)

# Tạo file config.toml
config_content = """
[server]
headless = true
port = 8501
enableCORS = false
enableXsrfProtection = false

[browser]
gatherUsageStats = false
"""

config_file = streamlit_dir / "config.toml"
with open(config_file, 'w') as f:
    f.write(config_content)

print("✅ Đã cấu hình Streamlit!")

## 🚀 Bước 6: Khởi động Admin Tool

In [ ]:
import subprocess
import threading
import time
from pyngrok import ngrok

# Kiểm tra admin tool có tồn tại không
admin_tool_path = "/content/SSGEpub/admin_tool/main.py"

if not os.path.exists(admin_tool_path):
    print(f"❌ Không tìm thấy admin tool tại {admin_tool_path}")
    print("Vui lòng kiểm tra repository đã được clone đúng chưa.")
else:
    print("🚀 Đang khởi động Admin Tool...")
    
    # Tạo ngrok tunnel
    try:
        public_url = ngrok.connect(8501)
        print(f"\n🌐 Admin Tool URL: {public_url}")
        print(f"📱 Truy cập URL trên để sử dụng admin tool")
        print(f"⏰ Tool sẽ khởi động trong vài giây...\n")
    except Exception as e:
        print(f"⚠️ Không thể tạo ngrok tunnel: {e}")
        print("Admin tool sẽ chạy trên localhost:8501")
    
    # Khởi động Streamlit
    !streamlit run {admin_tool_path} --server.port 8501 --server.headless true

## 🛠️ Troubleshooting

Nếu gặp lỗi, hãy thử:

1. **Restart Runtime**: Runtime > Restart runtime
2. **Kiểm tra repository**: Đảm bảo URL repository đúng
3. **Kiểm tra tokens**: GitHub token và ngrok token có đúng không
4. **Chạy lại từ đầu**: Chạy lại tất cả các cell

### Lệnh hữu ích:

In [ ]:
# Kiểm tra trạng thái
print("📊 Thông tin hệ thống:")
print(f"📂 Thư mục hiện tại: {os.getcwd()}")
print(f"📁 Admin tool: {'✅' if os.path.exists('/content/SSGEpub/admin_tool/main.py') else '❌'}")
print(f"🔑 GitHub token: {'✅' if os.environ.get('GITHUB_TOKEN') else '❌'}")

# Kiểm tra git status
if os.path.exists('/content/SSGEpub/.git'):
    %cd /content/SSGEpub
    !git status --short
else:
    print("❌ Không phải git repository")

In [ ]:
# Restart admin tool nếu cần
print("🔄 Restart Admin Tool")

# Kill existing processes
!pkill -f streamlit

# Wait a moment
import time
time.sleep(2)

# Start again
%cd /content/SSGEpub
!streamlit run admin_tool/main.py --server.port 8501 --server.headless true